In [ ]:
"""
Student Attendance System with CNN & OpenCV
============================================

A lightweight attendance system using:
- **CNN** for face recognition (M_2B_CNN.ipynb style)
- **OpenCV Haar Cascade** for multi-face detection
- **85% confidence threshold** to prevent false positives

This script has cell-style comments for easy conversion to Jupyter notebook.
"""

# Cell 1: Imports & Configuration
Import necessary libraries and set up configuration parameters.

In [ ]:
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
import csv

# TensorFlow/Keras imports
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TF warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Configuration - works in both .py and .ipynb
try:
    BASE_DIR = Path(__file__).parent.absolute()
except NameError:
    # For Jupyter notebooks - go up from notebooks/ folder
    BASE_DIR = Path('..').absolute() if Path('../dataset').exists() else Path('.').absolute()
DATASET_DIR = BASE_DIR / 'dataset'
MODEL_DIR = BASE_DIR / 'models'
ATTENDANCE_DIR = BASE_DIR / 'attendance_logs'

# Create directories if they don't exist
MODEL_DIR.mkdir(exist_ok=True)
ATTENDANCE_DIR.mkdir(exist_ok=True)

# Hyperparameters - OPTIMIZED FOR MEMORY EFFICIENCY
IMG_SIZE = 64  # Reduced from 160/224 to save memory
CHANNELS = 3
BATCH_SIZE = 16  # Smaller batch size for stability
EPOCHS = 30
VALIDATION_SPLIT = 0.2
RANDOM_STATE = 42
MIN_CONFIDENCE = 0.50  # 50% threshold - adjusted for real-world camera conditions
MIN_FACE_SIZE = 30  # Minimum face size for detection

print("=" * 50)
print("Student Attendance System - Configuration")
print("=" * 50)
print(f"TensorFlow Version: {tf.__version__}")
print(f"OpenCV Version: {cv2.__version__}")
print(f"Dataset Directory: {DATASET_DIR}")
print(f"Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Confidence Threshold: {MIN_CONFIDENCE * 100}%")
print("=" * 50)

# Cell 2: Data Preprocessing Functions
Functions for loading and preprocessing face images.

In [ ]:
def preprocess_image(img, target_size=(IMG_SIZE, IMG_SIZE)):
    """
    Preprocess a single image for CNN input.
    
    Args:
        img: BGR image (numpy array)
        target_size: Tuple of (height, width) for resizing
    
    Returns:
        Preprocessed image normalized to [0, 1]
    """
    if img is None:
        return None
    
    # Resize to target size
    img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
    
    # Convert BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Normalize to [0, 1]
    img = img.astype(np.float32) / 255.0
    
    return img


def check_image_quality(img):
    """
    Check if an image meets quality standards.
    
    Args:
        img: Image (numpy array)
    
    Returns:
        Tuple (is_valid, reason)
    """
    if img is None:
        return False, "Image is None"
    
    # Check minimum dimensions
    if img.shape[0] < 30 or img.shape[1] < 30:
        return False, "Image too small"
    
    # Convert to grayscale for quality checks
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    
    # Check blur using Laplacian variance
    blur_score = cv2.Laplacian(gray, cv2.CV_64F).var()
    if blur_score < 20:
        return False, f"Too blurry (score: {blur_score:.1f})"
    
    # Check contrast
    contrast = np.std(gray)
    if contrast < 10:
        return False, f"Low contrast (score: {contrast:.1f})"
    
    return True, "OK"


print("Data preprocessing functions loaded.")

# Cell 3: Haar Cascade Multi-Face Detection
Initialize OpenCV Haar Cascade for detecting multiple faces in a single frame.

In [ ]:
# Load Haar Cascade classifiers
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)
face_cascade_alt = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml'
)


def detect_multiple_faces(image, scale_factor=1.1, min_neighbors=5):
    """
    Detect ALL faces in an image using Haar Cascade.
    Supports detecting multiple faces simultaneously.
    
    Args:
        image: BGR image (numpy array)
        scale_factor: Scale factor for cascade
        min_neighbors: Minimum neighbors for detection
    
    Returns:
        List of (x, y, w, h) tuples for each detected face
    """
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply histogram equalization for better detection
    gray = cv2.equalizeHist(gray)
    
    # Detect faces using primary cascade
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=scale_factor,
        minNeighbors=min_neighbors,
        minSize=(MIN_FACE_SIZE, MIN_FACE_SIZE),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    
    # If no faces found, try alternative cascade
    if len(faces) == 0:
        faces = face_cascade_alt.detectMultiScale(
            gray,
            scaleFactor=scale_factor,
            minNeighbors=min_neighbors,
            minSize=(MIN_FACE_SIZE, MIN_FACE_SIZE),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
    
    return list(faces) if len(faces) > 0 else []


def extract_face(image, face_rect, margin=0.1):
    """
    Extract and preprocess a face from an image.
    
    Args:
        image: BGR image
        face_rect: (x, y, w, h) tuple
        margin: Margin around face as fraction
    
    Returns:
        Preprocessed face image or None
    """
    x, y, w, h = face_rect
    
    # Add margin
    x1 = max(0, int(x - w * margin))
    y1 = max(0, int(y - h * margin))
    x2 = min(image.shape[1], int(x + w + w * margin))
    y2 = min(image.shape[0], int(y + h + h * margin))
    
    # Extract face region
    face = image[y1:y2, x1:x2]
    
    if face.size == 0:
        return None
    
    return face


print("Haar Cascade face detection initialized.")
print(f"  - Primary cascade loaded: {not face_cascade.empty()}")
print(f"  - Alt cascade loaded: {not face_cascade_alt.empty()}")

# Cell 4: Dataset Loading
Load images from the dataset directory and prepare for training.

In [ ]:
def load_dataset(dataset_path, img_size=(IMG_SIZE, IMG_SIZE)):
    """
    Load and preprocess images from the dataset directory.
    
    Args:
        dataset_path: Path to dataset directory
        img_size: Target image size
    
    Returns:
        X (images), y (labels), class_names (list of student IDs)
    """
    images = []
    labels = []
    class_names = []
    
    dataset_path = Path(dataset_path)
    
    # Get all student directories (sorted for consistency)
    student_dirs = sorted([d for d in dataset_path.iterdir() if d.is_dir()])
    
    print(f"\nLoading dataset from: {dataset_path}")
    print(f"Found {len(student_dirs)} students:")
    
    for idx, student_dir in enumerate(student_dirs):
        student_id = student_dir.name
        class_names.append(student_id)
        
        # Get all image files
        image_files = list(student_dir.glob('*.jpg')) + list(student_dir.glob('*.png'))
        print(f"  [{idx}] {student_id}: {len(image_files)} images")
        
        for img_path in image_files:
            # Load image
            img = cv2.imread(str(img_path))
            
            if img is None:
                continue
            
            # Preprocess image
            img = preprocess_image(img, img_size)
            
            if img is not None:
                images.append(img)
                labels.append(idx)
    
    # Convert to numpy arrays
    X = np.array(images, dtype=np.float32)
    y = np.array(labels, dtype=np.int32)
    
    print(f"\nDataset loaded successfully!")
    print(f"  Total images: {len(X)}")
    print(f"  Image shape: {X.shape[1:]}")
    print(f"  Number of classes: {len(class_names)}")
    
    return X, y, class_names


# Load the dataset
print("\n" + "=" * 50)
print("Loading Dataset...")
print("=" * 50)
X, y, CLASS_NAMES = load_dataset(DATASET_DIR)
NUM_CLASSES = len(CLASS_NAMES)

# Cell 5: Train/Test Split
Split the dataset into training and testing sets.

In [ ]:
print("\n" + "=" * 50)
print("Splitting Dataset...")
print("=" * 50)

# Stratified split to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=VALIDATION_SPLIT,
    random_state=RANDOM_STATE,
    stratify=y
)

# Convert labels to categorical (one-hot encoding)
y_train_cat = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test_cat = keras.utils.to_categorical(y_test, NUM_CLASSES)

print(f"Training set: {len(X_train)} images")
print(f"Test set: {len(X_test)} images")
print(f"Number of classes: {NUM_CLASSES}")

# Cell 6: CNN Model Architecture
Build a lightweight CNN model following M_2B_CNN.ipynb style.

In [ ]:
def create_cnn_model(input_shape, num_classes):
    """
    Create a lightweight CNN model for face recognition.
    Architecture follows M_2B_CNN.ipynb style.
    
    Args:
        input_shape: Tuple of (height, width, channels)
        num_classes: Number of output classes
    
    Returns:
        Compiled Keras model
    """
    model = Sequential([
        # Input layer
        keras.Input(shape=input_shape),
        
        # Block 1: Conv -> Pool -> Dropout
        Conv2D(32, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Block 2: Conv -> Pool -> Dropout
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Flatten and Dense layers
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        
        # Output layer
        Dense(num_classes, activation='softmax')
    ])
    
    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


print("\n" + "=" * 50)
print("Building CNN Model...")
print("=" * 50)

# Create model
model = create_cnn_model((IMG_SIZE, IMG_SIZE, CHANNELS), NUM_CLASSES)
model.summary()

# Cell 7: Data Augmentation & Model Training
Set up data augmentation and train the model.

In [ ]:
print("\n" + "=" * 50)
print("Training Model...")
print("=" * 50)

# Data augmentation for better generalization
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    ),
    ModelCheckpoint(
        str(MODEL_DIR / 'best_model.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train_cat, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test_cat),
    callbacks=callbacks,
    verbose=1
)

# Save final model
model.save(str(MODEL_DIR / 'final_model.keras'))
print(f"\nModel saved to: {MODEL_DIR}")

# Cell 8: Training Visualization
Plot training history (accuracy and loss curves).

In [ ]:
print("\n" + "=" * 50)
print("Generating Training Plots...")
print("=" * 50)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Train Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Val Accuracy')
axes[0].set_title('Model Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# Loss plot
axes[1].plot(history.history['loss'], label='Train Loss')
axes[1].plot(history.history['val_loss'], label='Val Loss')
axes[1].set_title('Model Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(str(MODEL_DIR / 'training_history.png'), dpi=100)
plt.show()

print(f"Training plots saved to: {MODEL_DIR / 'training_history.png'}")

# Cell 9: Model Evaluation
Evaluate the model on the test set.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("\n" + "=" * 50)
print("Evaluating Model...")
print("=" * 50)

# Make predictions
y_pred_proba = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred_conf = np.max(y_pred_proba, axis=1)

# Overall accuracy
accuracy = np.mean(y_pred == y_test)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

# High confidence accuracy (with 85% threshold)
high_conf_mask = y_pred_conf >= MIN_CONFIDENCE
if high_conf_mask.sum() > 0:
    high_conf_acc = np.mean(y_pred[high_conf_mask] == y_test[high_conf_mask])
    print(f"High Confidence (>={MIN_CONFIDENCE*100:.0f}%) Accuracy: {high_conf_acc * 100:.2f}%")
    print(f"  Predictions above threshold: {high_conf_mask.sum()} / {len(y_test)}")
else:
    print("No predictions above confidence threshold!")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=CLASS_NAMES))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig(str(MODEL_DIR / 'confusion_matrix.png'), dpi=100)
plt.show()

print(f"Confusion matrix saved to: {MODEL_DIR / 'confusion_matrix.png'}")

# Cell 10: Multi-Face Recognition Function
Core function for detecting and recognizing multiple faces in a frame.

In [ ]:
def recognize_faces(image, model, class_names, confidence_threshold=MIN_CONFIDENCE):
    """
    Detect and recognize ALL faces in an image.
    Uses Haar Cascade for detection and CNN for recognition.
    
    Args:
        image: BGR image from camera or file
        model: Trained CNN model
        class_names: List of student IDs
        confidence_threshold: Minimum confidence for valid prediction
    
    Returns:
        List of (face_rect, student_id, confidence) tuples
    """
    results = []
    
    # Detect all faces using Haar Cascade
    faces = detect_multiple_faces(image)
    
    for face_rect in faces:
        # Extract face region
        face = extract_face(image, face_rect)
        
        if face is None:
            continue
        
        # Check face quality
        is_valid, reason = check_image_quality(face)
        if not is_valid:
            continue
        
        # Preprocess for CNN
        face_preprocessed = preprocess_image(face, (IMG_SIZE, IMG_SIZE))
        face_input = np.expand_dims(face_preprocessed, axis=0)
        
        # Predict
        predictions = model.predict(face_input, verbose=0)[0]
        pred_class = np.argmax(predictions)
        confidence = predictions[pred_class]
        
        # Only accept high confidence predictions (no false positives)
        if confidence >= confidence_threshold:
            student_id = class_names[pred_class]
        else:
            student_id = "Unknown"
        
        results.append((face_rect, student_id, confidence))
    
    return results


print("\nMulti-face recognition function ready!")
print(f"  - Confidence threshold: {MIN_CONFIDENCE * 100}%")
print(f"  - Classes: {CLASS_NAMES}")

# Cell 11: Attendance Tracking Class
Class for tracking student attendance and preventing duplicates.

In [ ]:
class AttendanceTracker:
    """
    Track student attendance with duplicate prevention.
    """
    
    def __init__(self, class_names, cooldown_seconds=60):
        self.class_names = class_names
        self.cooldown = cooldown_seconds
        self.attendance = {}  # {student_id: (first_seen, last_seen, count)}
        self.session_start = datetime.now()
    
    def mark_present(self, student_id, confidence):
        """
        Mark a student as present.
        
        Args:
            student_id: Student identifier
            confidence: Prediction confidence
        
        Returns:
            True if newly marked, False if already marked
        """
        if student_id == "Unknown":
            return False
        
        now = datetime.now()
        
        if student_id in self.attendance:
            first_seen, last_seen, count = self.attendance[student_id]
            # Check cooldown
            if (now - last_seen).seconds < self.cooldown:
                self.attendance[student_id] = (first_seen, now, count)
                return False
            self.attendance[student_id] = (first_seen, now, count + 1)
        else:
            self.attendance[student_id] = (now, now, 1)
            print(f"✓ {student_id} marked PRESENT (confidence: {confidence:.1%})")
        
        return True
    
    def get_summary(self):
        """Get attendance summary string."""
        present = len(self.attendance)
        total = len(self.class_names)
        return f"Present: {present}/{total}"
    
    def save_to_csv(self, filepath=None):
        """
        Save attendance to CSV file.
        
        Args:
            filepath: Optional custom path
        
        Returns:
            Path to saved file
        """
        if filepath is None:
            timestamp = self.session_start.strftime("%Y%m%d_%H%M%S")
            filepath = ATTENDANCE_DIR / f'attendance_{timestamp}.csv'
        
        with open(filepath, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Student_ID', 'First_Seen', 'Last_Seen', 'Detections', 'Status'])
            
            for student_id in self.class_names:
                if student_id in self.attendance:
                    first, last, count = self.attendance[student_id]
                    writer.writerow([
                        student_id,
                        first.strftime('%H:%M:%S'),
                        last.strftime('%H:%M:%S'),
                        count,
                        'Present'
                    ])
                else:
                    writer.writerow([student_id, '', '', 0, 'Absent'])
        
        print(f"\nAttendance saved to: {filepath}")
        return str(filepath)


print("Attendance tracking system ready!")

# Cell 12: Real-Time Attendance System (Optional)
Run the camera-based attendance system.

In [ ]:
def run_realtime_attendance(model, class_names, camera_id=0, duration_seconds=None):
    """
    Run real-time attendance system using camera.
    
    Controls:
        'q' - Quit and save attendance
        's' - Save attendance immediately
    
    Args:
        model: Trained CNN model
        class_names: List of student IDs
        camera_id: Camera device ID
        duration_seconds: Auto-stop after N seconds (None for manual)
    
    Returns:
        AttendanceTracker object with results
    """
    print("\n" + "=" * 50)
    print("REAL-TIME ATTENDANCE SYSTEM")
    print("=" * 50)
    print("Controls: 'q' = quit | 's' = save")
    print(f"Confidence threshold: {MIN_CONFIDENCE * 100}%")
    print("=" * 50 + "\n")
    
    # Open camera
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print("ERROR: Could not open camera!")
        return None
    
    # Initialize tracker
    tracker = AttendanceTracker(class_names)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Recognize faces
        results = recognize_faces(frame, model, class_names)
        
        # Draw results
        for face_rect, student_id, confidence in results:
            x, y, w, h = face_rect
            tracker.mark_present(student_id, confidence)
            
            # Simple color: green for recognized, red for unknown
            color = (0, 255, 0) if student_id != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            
            # Label
            label = f"{student_id}: {confidence:.0%}"
            cv2.putText(frame, label, (x, y - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # Simple status
        cv2.putText(frame, tracker.get_summary(), (10, 25),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow('Attendance', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            tracker.save_to_csv()
            break
        elif key == ord('s'):
            tracker.save_to_csv()
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"\nFinal: {tracker.get_summary()}")
    return tracker


print("\n" + "=" * 50)
print("TRAINING COMPLETE!")
print("=" * 50)
print("\nTo test with camera, uncomment and run:")
print("  tracker = run_realtime_attendance(model, CLASS_NAMES)")
print("\nTo test with an image:")
print("  results = recognize_faces(cv2.imread('path/to/image.jpg'), model, CLASS_NAMES)")
print("  for face_rect, name, conf in results:")
print("      print(f'{name}: {conf:.1%}')")

In [ ]:
if __name__ == "__main__":
    tracker = run_realtime_attendance(model, CLASS_NAMES)